# Modelling Passenger Rail Reliability with MCRoute

In [8]:
import sys
import math
import pandas as pd
import altair as alt
import networkx as nx
import numpy as np

sys.path.append(r"..")

from mcroute import Network, StateSpace
import mcroute.matrix as matrix
import mcroute.vector as vector

In [9]:
model_dict = {}
trains =  [84, 85, 87, 88]
# We wil use the same state space for all four models
state_space = StateSpace.from_csv('data/via/states.csv')
# Have an identity matrix ready and made for our nodes
identity = matrix.identity(state_space)

for t in trains:
    print(f"Preparing Train {t} model")
    # Create a network for each model
    n = Network(state_space)
    edge_stats = pd.read_csv(f"data/via/train_{t}_edges.csv")
    # Let's create our nodes first
    nodes = set(edge_stats['from'].tolist()).union(set(edge_stats['to'].tolist()))
    for node in nodes:
        n.add_node(node, identity)
    
    # Now let's edd the edges
    for idx, edge in edge_stats.iterrows():
        mx = matrix.truncnorm(state_space, mean=0, std=edge['std'])
        n.add_edge(edge['from'], edge['to'], mx)

    # Add it to our model dict
    model_dict[t] = n

Preparing Train 84 model
Preparing Train 85 model
Preparing Train 87 model
Preparing Train 88 model


In [10]:
# path = nx.shortest_path(model_dict[85], source="Toronto", target="London")
# model_dict[85].trajectories(path, vector.unit(state_space, '0'), n=20)

## Visualize Traversal Statistics

In [11]:
stats = []
p0 = vector.unit(state_space, '0')
for t in trains:
    if t % 2 == 0:
        path = nx.shortest_path(model_dict[t], source="London", target="Toronto")
    else:
        path = nx.shortest_path(model_dict[t], source="Toronto", target="London")
    vecs = model_dict[t].traverse(path, p0)
    means = []
    stds = []
    for v in vecs:
        average = np.average(state_space.values, weights=v)
        # Fast and numerically precise:
        variance = np.average((state_space.values-average)**2, weights=v)
        std = math.sqrt(variance)
        means.append(average)
        stds.append(std)
    df = pd.DataFrame(zip(means, stds), columns=['mean', 'std']).reset_index()
    df.columns = ['element', 'mean', 'std']
    df['train'] = t
    stats.append(df)
allstats = pd.concat(stats)
allstats

,element,mean,std,train
0,0,0.000000,0.000000,84
1,1,0.000000,0.000000,84
2,2,0.500009,6.975712,84
3,3,0.500009,6.975712,84
4,4,1.000040,7.418201,84
...,...,...,...,...
10,10,2.830300,18.147323,88
11,11,2.830300,18.147323,88
12,12,3.351006,18.052266,88
13,13,3.351006,18.052266,88


In [12]:
alt.Chart(allstats).mark_line().encode(
    alt.X('element:O', title='Step'),
    alt.Y('mean:Q', title='Modelled Mean Schedule Deviation'),
    color=alt.Color('train:N', title='Train')
).properties(
    width = 800,
    height=300
).configure(font='Raleway').configure_view(strokeWidth=0).configure_axis(grid=False, domain=False)

alt.Chart(...)

In [13]:
alt.Chart(allstats).mark_line().encode(
    alt.X('element:O', title='Step'),
    alt.Y('std:Q', title='Modelled Standard Deviation of Schedule Deviation'),
    color=alt.Color('train:N', title='Train')
).properties(
    width = 800,
    height=300
).configure(font='Raleway').configure_view(strokeWidth=0).configure_axis(grid=False, domain=False)

alt.Chart(...)

In [14]:
new_model_dict = {}
for t in trains:
    print(f"Preparing Train {t} model")
    # Create a network for each model
    n = Network(state_space)
    edge_stats = pd.read_csv(f"data/via/train_{t}_edges.csv")
    # Let's create our nodes first
    nodes = set(edge_stats['from'].tolist()).union(set(edge_stats['to'].tolist()))
    for node in nodes:
        n.add_node(node, identity)
    
    # Now let's edd the edges
    for idx, edge in edge_stats.iterrows():
        if (edge['from'] == "Brampton" and edge['to'] == "Georgetown") or (edge['to'] == "Brampton" and edge['from'] == "Georgetown"):
            print("Crossover Link")
            mx = matrix.truncnorm(state_space, mean=0, std=edge['std']*0.5)
        else:
            mx = matrix.truncnorm(state_space, mean=0, std=edge['std'])
        n.add_edge(edge['from'], edge['to'], mx)

    # Add it to our model dict
    new_model_dict[t] = n

Preparing Train 84 model
Crossover Link
Preparing Train 85 model
Crossover Link
Preparing Train 87 model
Crossover Link
Preparing Train 88 model
Crossover Link


In [15]:
stats = []
p0 = vector.unit(state_space, '0')
for t in trains:
    if t % 2 == 0:
        path = nx.shortest_path(new_model_dict[t], source="London", target="Toronto")
    else:
        path = nx.shortest_path(new_model_dict[t], source="Toronto", target="London")
    vecs = new_model_dict[t].traverse(path, p0)
    means = []
    stds = []
    for v in vecs:
        average = np.average(state_space.values, weights=v)
        # Fast and numerically precise:
        variance = np.average((state_space.values-average)**2, weights=v)
        std = math.sqrt(variance)
        means.append(average)
        stds.append(std)
    df = pd.DataFrame(zip(means, stds), columns=['mean', 'std']).reset_index()
    df.columns = ['element', 'mean', 'std']
    df['train'] = t
    stats.append(df)
new_allstats = pd.concat(stats)
new_allstats
all_allstats = pd.merge(allstats, new_allstats, on=['element', 'train'])
all_allstats.columns = ['element', 'Mean', 'Standard Deviation', 'train', 'New Mean', 'New Standard Deviation']
all_allstats

,element,Mean,Standard Deviation,train,New Mean,New Standard Deviation
0,0,0.000000,0.000000,84,0.000000,0.000000
1,1,0.000000,0.000000,84,0.000000,0.000000
2,2,0.500009,6.975712,84,0.500009,6.975712
3,3,0.500009,6.975712,84,0.500009,6.975712
4,4,1.000040,7.418201,84,1.000040,7.418201
...,...,...,...,...,...,...
61,10,2.830300,18.147323,88,2.830300,18.147323
62,11,2.830300,18.147323,88,2.830300,18.147323
63,12,3.351006,18.052266,88,3.342210,18.072006
64,13,3.351006,18.052266,88,3.342210,18.072006


In [16]:
plot_means = all_allstats[['element', 'train', 'Mean', 'New Mean']].copy()
plot_means.columns = ['element', 'train', 'Before', 'After']
plot_means = plot_means.melt(id_vars=['element', 'train'])
alt.Chart(plot_means).mark_line().encode(
    alt.X('element:O', title='Step'),
    alt.Y('value:Q', title='Mean (min)'),
    color=alt.Color('train:N', title='Train'),
    strokeDash = alt.StrokeDash('variable:N', title='Scenario', sort='-x')
).properties(
    width = 800,
    height=200,
    title='Modelled Mean Schedule Deviaition Before and After Reliability Improvements'
).configure(font='Raleway').configure_view(strokeWidth=0).configure_axis(grid=False, domain=False)

alt.Chart(...)

In [18]:
plot_stds = all_allstats[['element', 'train', 'Standard Deviation', 'New Standard Deviation']].copy()
plot_stds.columns = ['element', 'train', 'Before', 'After']
plot_stds = plot_stds.melt(id_vars=['element', 'train'])
alt.Chart(plot_stds).mark_line().encode(
    alt.X('element:O', title='Step'),
    alt.Y('value:Q', title='Standard Deviaiton (min)'),
    color=alt.Color('train:N', title='Train'),
    strokeDash = alt.StrokeDash('variable:N', sort='-x', title='Scenario')
).properties(
    width = 800,
    height=400,
    title='Modelled Standard Deviation of Schedule Deviaition Before and After Reliability Improvements'
).configure(font='Raleway').configure_view(strokeWidth=0).configure_axis(grid=False, domain=False)

alt.Chart(...)